# Feature engineering

**Purpose of script:**

Creating new features 

- In: dataframe_plain
- Out: dataframe_extended (with additional feature columns)

In [1]:
import xarray
import rasterio
import gemgis as gg
import pandas as pd
import numpy as np
from tqdm import tqdm
from astral.sun import sun
from astral import LocationInfo
import pyproj

import datetime as dt

from os import listdir
from os.path import isfile, join

##### Relevant paths

In [2]:
path_dataframe_plain = r"../Data/combined/dataframe_plain/"
mw_path = r"../Data/microwave-rs/mw_interpolated/"
path_elevation =  r"../Data/elevation_data/gimpdem_1km_compressed.tif"
out_path = r"../Data/combined/dataframe_extended/"

## Features

##### Row and column

In [7]:
def add_row_and_col(df):
        # add row and column features:
        df['col'] = df.groupby("x").ngroup() # xshape 2663 
        df['row'] = df.groupby("y").ngroup(ascending=False) # yshape 1462
        return df

##### Date

In [29]:
def add_date_year(df, melt_date):
    date = pd.to_datetime(melt_date).date()
    df['date'] = date
    df['year'] = date.year
    return df

##### Aggregated/pooled values

In [9]:
from typing import Union
from typing import Tuple
from scipy.stats import mode
from scipy.signal import convolve2d

def get_window(image: np.ndarray, window_size: int, center: Tuple[int, int]) -> np.ndarray:
    top = max(center[0] - window_size // 2, 0)
    bottom = min(center[0] + window_size // 2 + 1, image.shape[0])
    left = max(center[1] - window_size // 2, 0)
    right = min(center[1] + window_size // 2 + 1, image.shape[1])
    window = image[top:bottom, left:right]
    return window


# need to fix? : only calculate if the middle value is not nan - else all nan columns around 1 and 0 are going to have a value.

def convolve(image, window_size, convolution_fn: Union['mean', 'min', 'max', 'sum']):
    image = image[0].values
    image[image == -1] = np.nan
    
    if convolution_fn == 'mean':
        kernel = np.ones((window_size, window_size))  # kernel for mean convolution
        result = np.zeros_like(image, dtype=np.float64)
        # Compute the sum and count of non-NaN values in the kernel window
        counts = convolve2d(~np.isnan(image), kernel, mode='same', boundary='fill', fillvalue=0)
        sums = convolve2d(np.nan_to_num(image), kernel, mode='same', boundary='fill', fillvalue=0)
        # Calculate the mean, ignoring NaN values
        result[counts > 0] = sums[counts > 0] / counts[counts > 0]
        # Set the output to NaN where all values in the kernel window are NaN
        result[counts == 0] = np.nan
        return result
        
    elif convolution_fn == 'max':
        result = np.zeros_like(image, dtype=np.float64)
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                window = get_window(image, window_size, (i, j))
                non_nan_values = window[~np.isnan(window)]
                if len(non_nan_values) == 0:
                    result[i, j] = np.nan
                else:
                    result[i, j] = np.nanmax(non_nan_values)

    elif convolution_fn == 'min':
        result = np.zeros_like(image, dtype=np.float64)
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                window = get_window(image, window_size, (i, j))
                non_nan_values = window[~np.isnan(window)]
                if len(non_nan_values) == 0:
                    result[i, j] = np.nan
                else:
                    result[i, j] = np.nanmin(non_nan_values)
        return result

    # elif convolution_fn == 'maxdiff':
    #     result = np.zeros_like(image, dtype=np.float64)
    #     for i in range(image.shape[0]):
    #         for j in range(image.shape[1]):
    #             window = get_window(image, window_size, (i, j))
    #             non_nan_values = window[~np.isnan(window)]
    #             if len(non_nan_values) == 0:
    #                 result[i, j] = np.nan
    #             else:
    #                 result[i, j] = np.nanmax(non_nan_values) - np.nanmin(non_nan_values)
    #     return result

    elif convolution_fn == 'sum':
        result = np.zeros_like(image, dtype=np.float64)
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                window = get_window(image, window_size, (i, j))
                non_nan_values = window[~np.isnan(window)]
                if len(non_nan_values) == 0:
                    result[i, j] = np.nan
                else:
                    result[i, j] = np.nansum(non_nan_values)
        return result
        
    else: 
        print('not available function')
    return

In [14]:
def convolution_to_df(convolution_raster, column_name):
    nrows, ncols = convolution_raster.shape
    # create an array of x and y positions
    x = np.tile(np.arange(ncols), nrows)
    y = np.repeat(np.arange(nrows), ncols)
    # create a DataFrame with x, y, and pixel values as columns
    df = pd.DataFrame({'col': x, 'row': y, column_name: convolution_raster.flatten()})
    return df 

##### Elevation data

In [10]:
def add_elevation(data):
    df = data.to_dataframe()
    df = df.reset_index()
    df = df[['x', 'y', 'band_data']]
    df.rename({'band_data': 'elevation_data'}, axis=1, inplace=True)
    return df

##### Slope

Slope is given as degree of incline angle: 0 means flat (no slope == horizontal), 90 means (most possible slope == vertical)

In [11]:
def get_slope(data):
    slope = gg.raster.calculate_slope(data)
    nrows, ncols = slope.shape
    # create an array of x and y positions
    x = np.tile(np.arange(ncols), nrows)
    y = np.repeat(np.arange(nrows), ncols)
    # create a DataFrame with x, y, and pixel values as columns
    df_slope = pd.DataFrame({'col': x, 'row': y, 'slope_data': slope.flatten()})
    return df_slope

##### Aspect

Aspect is given as degree of direction slope faces: 0 means facing North, 90 means facing East, 180 means facing South, 270 means facing West

In [12]:
def get_aspect(data):
    aspect = gg.raster.calculate_aspect(data)
    nrows, ncols = aspect.shape
    # create an array of x and y positions
    x = np.tile(np.arange(ncols), nrows)
    y = np.repeat(np.arange(nrows), ncols)
    # create a DataFrame with x, y, and pixel values as columns
    df_aspect = pd.DataFrame({'col': x, 'row': y, 'aspect_data': aspect.flatten()})
    return df_aspect

##### Distance from margin/shore

In [10]:
# tbd
# add if coast column - if at least one na but not all 

##### Get solar duration

## Main:

In [4]:
def get_files(mw_path, path_dataframe_plain):
    # get plain files:
    df_plain_files = [f for f in listdir(path_dataframe_plain) if isfile(join(path_dataframe_plain, f))]
    # microwave files:
    mw_files = [f for f in listdir(mw_path) if isfile(join(mw_path, f))]
    return  mw_files, df_plain_files

In [30]:
def main(mw_files_list, df_plain_files_list, path_elevation, out_path, write = False):
    # get plain files:
    df_plain_files = df_plain_files_list
    # microwave files:
    mw_files = mw_files_list
    # load elevation data:
    data_elevation_xarray = xarray.open_dataarray(path_elevation)
    data_elevation_rasterio = rasterio.open(path_elevation)   

    for df_file in df_plain_files:
        melt_date =  df_file[5:15]
        print(melt_date)
        for mw_file in mw_files:
            if mw_file.startswith(melt_date):
                data_mw = xarray.open_dataarray(mw_path + mw_file)
                df = pd.read_parquet(path_dataframe_plain + df_file)
                # add row and column features:
                df = add_row_and_col(df)
                # get convolutions:
                df_conv_mean_3 = convolution_to_df(convolve(data_mw, 3, 'mean'), 'mean_3')
                df_conv_mean_9 = convolution_to_df(convolve(data_mw, 9, 'mean'), 'mean_9')
                #df_conv_maxdisff_5 = convolution_to_df(convolve(data_mw, 5, 'maxdiff'), 'maxdiff_9')
                df_conv_sum_5 = convolution_to_df(convolve(data_mw, 5, 'sum'), 'sum_5')
                # merge convolution:
                df_combined = pd.merge(df, df_conv_mean_3, how = 'left', on = ['row', 'col'])
                df_combined = pd.merge(df_combined, df_conv_mean_9, how = 'left', on = ['row', 'col'])
                #df_combined = pd.merge(df_combined, df_conv_maxdisff_5, how = 'left', on = ['row', 'col'])
                df_combined = pd.merge(df_combined, df_conv_sum_5, how = 'left', on = ['row', 'col'])
                # remove water in mw:
                df_combined = df_combined.loc[df_combined['mw_value'] != -1]
                # add date:
                df = add_date_year(df_combined, melt_date)
                # add elevation data:
                df_elevation = add_elevation(data_elevation_xarray)
                # get slope data:
                df_slope = get_slope(data_elevation_rasterio)
                # merge slope data:
                df_elevation = pd.merge(df_elevation, df_slope[["slope_data"]], how="left", right_index=True, left_index=True)
                # get aspect data:
                df_aspect = get_aspect(data_elevation_rasterio)
                # merge aspect data:
                df_elevation = pd.merge(df_elevation, df_aspect[["aspect_data"]], how="left", right_index=True, left_index=True)
                # merge elevation data:
                df_with_elevation = pd.merge(df, df_elevation, how = 'left', on = ['y', 'x'])              
                
                # write to parquet:
                if write == True:
                    df_with_elevation.to_parquet(out_path + 'melt_'+ melt_date + '_extended.parquet.gzip', index= False)                    
    return df_with_elevation
                
                

Main

In [ ]:
#main(mw_path, path_dataframe_plain, path_elevation, out_path)
main(*get_files(mw_path, path_dataframe_plain), path_elevation, out_path)

## Testing

### Testing Lina

In [31]:
df = main(['2019-06-08_mw.tif'], ['melt_2019-06-08.parquet.gzip'], path_elevation, out_path)  

2019-06-08


In [32]:
df

,x,y,mw_value,opt_value,col,row,mean_3,mean_9,sum_5,date,year,elevation_data,slope_data,aspect_data
0,-636500.00,-662500.00,0.00,-1.00,0,0,0.00,0.00,0.00,2019-06-08,2019,14.00,0.00,0.00
1,-635500.00,-662500.00,0.00,-1.00,1,0,0.00,0.00,0.00,2019-06-08,2019,14.00,0.00,0.00
2,-634500.00,-662500.00,0.00,-1.00,2,0,0.00,0.00,0.00,2019-06-08,2019,14.00,0.00,0.00
3,-633500.00,-662500.00,0.00,-1.00,3,0,0.00,0.00,0.00,2019-06-08,2019,14.00,0.00,0.00
4,-632500.00,-662500.00,0.00,-1.00,4,0,0.00,0.00,0.00,2019-06-08,2019,14.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2278720,58500.00,-3324500.00,0.00,-1.00,695,2662,0.00,0.00,0.00,2019-06-08,2019,44.00,6.48,88.99
2278721,59500.00,-3324500.00,0.00,-1.00,696,2662,0.00,0.00,0.00,2019-06-08,2019,44.00,0.63,0.00
2278722,60500.00,-3324500.00,0.00,-1.00,697,2662,0.00,0.00,0.00,2019-06-08,2019,44.00,0.04,315.00
2278723,61500.00,-3324500.00,0.00,-1.00,698,2662,0.00,0.00,0.00,2019-06-08,2019,45.00,0.03,270.00


In [103]:
# # for testing values around the 0-1 change in the data:

# tt = data_mw.values
# # indices = np.where(tt == 1)
# tt[0][74:80, 622:628]

### Testing Nina

In [64]:
# Define the source and destination coordinate reference systems
src_crs = pyproj.CRS.from_epsg(3413)  # WGS84 (longitude, latitude)
dst_crs = pyproj.CRS.from_epsg(4326)  # Web Mercator (used by most online maps)

# Define the transformer object
transformer = pyproj.Transformer.from_crs(src_crs, dst_crs)
# Convert all coordinates at once
lats, longs = transformer.transform(df["x"], df["y"])
# Define the location object
locations = [LocationInfo(lat, lon) for lat, lon in zip(lats, longs)]
# Parse all dates at once
dates = pd.to_datetime(df["date"])

# Define a function to calculate solar duration for a single location and date
def get_solar_duration(location, date):
    s = sun(location.observer, date=date)
    return (s['sunset'] - s['sunrise']).seconds / 60

obs = ephem.Observer()
sun = ephem.Sun()
# Apply the function to all locations and dates using vectorized operations
df["solar_duration"] = [get_solar_duration(loc, date) for loc, date in zip(locations, dates)]
df

,x,y,mw_value,opt_value,col,row,mean_3,mean_9,sum_5,date,year,elevation_data,slope_data,aspect_data,lat,lon,solar_duration
0,-636500.00,-662500.00,0.00,-1.00,0,0,0.00,0.00,0.00,2019-06-08,2019,14.00,0.00,0.00,81.53,-88.85,988.72
1,-635500.00,-662500.00,0.00,-1.00,1,0,0.00,0.00,0.00,2019-06-08,2019,14.00,0.00,0.00,81.54,-88.81,988.72
2,-634500.00,-662500.00,0.00,-1.00,2,0,0.00,0.00,0.00,2019-06-08,2019,14.00,0.00,0.00,81.55,-88.76,988.72
3,-633500.00,-662500.00,0.00,-1.00,3,0,0.00,0.00,0.00,2019-06-08,2019,14.00,0.00,0.00,81.55,-88.72,988.72
4,-632500.00,-662500.00,0.00,-1.00,4,0,0.00,0.00,0.00,2019-06-08,2019,14.00,0.00,0.00,81.56,-88.67,988.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2278720,58500.00,-3324500.00,0.00,-1.00,695,2662,0.00,0.00,0.00,2019-06-08,2019,44.00,6.48,88.99,59.98,-43.99,988.72
2278721,59500.00,-3324500.00,0.00,-1.00,696,2662,0.00,0.00,0.00,2019-06-08,2019,44.00,0.63,0.00,59.98,-43.97,988.72
2278722,60500.00,-3324500.00,0.00,-1.00,697,2662,0.00,0.00,0.00,2019-06-08,2019,44.00,0.04,315.00,59.98,-43.96,988.72
2278723,61500.00,-3324500.00,0.00,-1.00,698,2662,0.00,0.00,0.00,2019-06-08,2019,45.00,0.03,270.00,59.98,-43.94,988.72


In [76]:
# Define the source and destination coordinate reference systems
src_crs = pyproj.CRS.from_epsg(3413)  # WGS84 (longitude, latitude)
dst_crs = pyproj.CRS.from_epsg(4326)  # Web Mercator (used by most online maps)

# Define the transformer object
transformer = pyproj.Transformer.from_crs(src_crs, dst_crs)


def calculate_solar_duration(x, y, date):

      # Transform a pair of coordinates
      lat, lon = transformer.transform(x, y)

      # Define the suntime object
      location = LocationInfo(lat, lon)
      print(lat, lon)
      
      # specify the date for which to calculate sunrise and sunset
      date = dt.datetime.strptime(date, "%Y-%m-%d").date()

      # calculate the sunrise and sunset times
      s = sun(location.observer, date=date)
      sunrise = s['sunrise'].strftime('%H:%M:%S')
      sunset = s['sunset'].strftime('%H:%M:%S')
      print(sunrise)
      print(sunset)

      # calculate the solar duration
      duration = s['sunset'] - s['sunrise']

      # print the results
      print(f"Solar Duration in minutes: {duration.seconds / 60}")

      return duration.seconds / 60

calculate_solar_duration(-636500.00		, -662500.00	, "2019-06-08")

81.53389147728227 -88.85335536270664


TypeError: 'Sun' object is not callable

In [77]:
import ephem
from datetime import datetime

# define location
obs = ephem.Observer()
obs.lat = 81.53389147728227
obs.lon = -88.85335536270664

# define date
date = datetime(2019, 6, 8)

# calculate sunrise and sunset
sun = ephem.Sun()
sun.compute(obs)
sunrise = obs.previous_rising(sun, use_center=True).datetime().strftime("%H:%M:%S")
sunset = obs.next_setting(sun, use_center=True).datetime().strftime("%H:%M:%S")

print(f"Sunrise: {sunrise}")
print(f"Sunset: {sunset}")
print(f"Duration: {datetime.strptime(sunset, '%H:%M:%S') - datetime.strptime(sunrise, '%H:%M:%S')}")

Sunrise: 09:29:08
Sunset: 21:43:30
Duration: 12:14:22


In [160]:
import pytz

obs = ephem.Observer()
obs.lat = 64.1743
obs.lon = 51.7373
obs.date = datetime(2023, 1, 9)

sun = ephem.Sun()
sun.compute(obs)
sunrise = obs.next_rising(sun).datetime()
sunset = obs.next_setting(sun).datetime()

print(f"Sunrise: {sunrise}")
print(f"Sunset: {sunset}")
print(f"Duration: {sunset- sunrise}")

NeverUpError: 'Sun' is below the horizon at 2023/1/9 06:29:34

In [156]:
type(obs.date)

ephem.Date